In [ ]:
from ultralytics import YOLO
import cv2
import math
import serial
import time

# התחברות לארדואינו
try:
    arduino = serial.Serial(port='COM9', baudrate=9600, timeout=.1)
    print("Connected to Arduino on COM8")
except Exception as e:
    print(f"Error connecting to Arduino: {e}")
    arduino = None

# פונקציה לשליחת פקודה לארדואינו
def send_command(command):
    """שליחת פקודה לארדואינו דרך Serial"""
    if arduino:
        try:
            print(f"Sending command to Arduino: {command.strip()}")
            arduino.write((command + "\n").encode())
            time.sleep(0.1)  # השהייה ממושכת יותר
        except Exception as e:
            print(f"Error sending command to Arduino: {e}")
    else:
        print("Arduino is not connected. Command not sent.")

# אתחול YOLO
model = YOLO("yolo-Weights/yolov8n.pt")

# הפעלת מצלמה
cap = cv2.VideoCapture(2)
cap.set(3, 640)  # רוחב הפריים
cap.set(4, 480)  # גובה הפריים

# רשימת אובייקטים שניתן לזהות
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

# הגדרת אובייקטים רלוונטיים למעקב
relevant_objects = ["bird"]

# הגדרת משתנה למעקב אחרי אם זוהה אדם
person_detected = False
last_command = ""

while True:
    success, img = cap.read()
    
    if not success:  # אם לא הצלחנו לקרוא פריים מהמצלמה, נעצור את הלולאה
        print("Error reading frame from camera.")
        break

    results = model(img, stream=True)

    detected_command = "STOP"

    person_detected = False

    for r in results:
        boxes = r.boxes  # חילוץ תיבות זיהוי

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            class_name = classNames[cls]
            print(f"Detected: {class_name}, Confidence: {confidence}")

            if class_name in relevant_objects and confidence > 0.5:
                person_detected = True
                center_x = (x1 + x2) // 2
                frame_center_x = img.shape[1] // 2  # מרכז המסך בציר X

                if center_x < frame_center_x - 50:
                    detected_command = "LEFT"
                elif center_x > frame_center_x + 50:
                    detected_command = "RIGHT"
                else:
                    detected_command = "FORWARD"

                org = (x1, y1)
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                color = (255, 0, 0)  # צבע טקסט
                thickness = 2
                cv2.putText(img, f"{class_name} {confidence}", org, font, fontScale, color, thickness)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

    # אם לא זוהה אדם, שלח פקודות RIGHT ו-LEFT לסירוגין
    if not person_detected:
        if time.time() % 2 < 1:
            detected_command = "RIGHT"
        else:
            detected_command = "LEFT"

    # שלח פקודה רק אם היא שונה מהפקודה הקודמת
    if detected_command != last_command:
        send_command(detected_command)
        last_command = detected_command

    cv2.imshow('Webcam', img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Exiting program...")
        send_command("STOP")
        break

# ניקוי משאבים
cap.release()
cv2.destroyAllWindows()
